In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from itertools import product
from bokeh.io import show, output_notebook
from bokeh.layouts import gridplot
from bokeh.models import (BasicTicker, Circle, ColumnDataSource,
                          DataRange1d, Grid, LassoSelectTool, LinearAxis,
                          PanTool, Plot, ResetTool, WheelZoomTool)
from bokeh.sampledata.penguins import data
from bokeh.transform import factor_cmap
output_notebook()

Loading BokehJS ...

In [2]:
df = data.copy()
df["body_mass_kg"] = df["body_mass_g"] / 1000

SPECIES = sorted(df.species.unique())
COLUMNS = ("bill_length_mm", "bill_depth_mm", "body_mass_kg")
N = len(ATTRS)
df

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex,body_mass_kg
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,MALE,3.75
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,FEMALE,3.80
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,FEMALE,3.25
3,Adelie,Torgersen,NaN,NaN,NaN,NaN,NaN,NaN
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,FEMALE,3.45
...,...,...,...,...,...,...,...,...
339,Gentoo,Biscoe,NaN,NaN,NaN,NaN,NaN,NaN
340,Gentoo,Biscoe,46.8,14.3,215.0,4850.0,FEMALE,4.85
341,Gentoo,Biscoe,50.4,15.7,222.0,5750.0,MALE,5.75
342,Gentoo,Biscoe,45.2,14.8,212.0,5200.0,FEMALE,5.20


In [3]:
source = ColumnDataSource(data=df)

xdrs = [DataRange1d(bounds=None) for _ in range(N)]
ydrs = [DataRange1d(bounds=None) for _ in range(N)]

plots = []

for i, (y, x) in enumerate(product(ATTRS, reversed(ATTRS))):
    p = Plot(x_range=xdrs[i%N], y_range=ydrs[i//N],
             background_fill_color="#fafafa",
             border_fill_color="white", width=200, height=200, min_border=5)

    if i % N == 0:  # first column
        p.min_border_left = p.min_border + 4
        p.width += 40
        yaxis = LinearAxis(axis_label=y)
        yaxis.major_label_orientation = "vertical"
        p.add_layout(yaxis, "left")
        yticker = yaxis.ticker
    else:
        yticker = BasicTicker()
    p.add_layout(Grid(dimension=1, ticker=yticker))

    if i >= N*(N-1):  # last row
        p.min_border_bottom = p.min_border + 40
        p.height += 40
        xaxis = LinearAxis(axis_label=x)
        p.add_layout(xaxis, "below")
        xticker = xaxis.ticker
    else:
        xticker = BasicTicker()
    p.add_layout(Grid(dimension=0, ticker=xticker))

    circle = Circle(x=x, y=y, fill_alpha=0.6, size=5, line_color=None,
                    fill_color=factor_cmap('species', 'Category10_3', SPECIES))
    r = p.add_glyph(source, circle)
    p.x_range.renderers.append(r)
    p.y_range.renderers.append(r)

    # suppress the diagonal
    if (i%N) + (i//N) == N-1:
        r.visible = False
        p.grid.grid_line_color = None

    p.add_tools(PanTool(), WheelZoomTool(), ResetTool(), LassoSelectTool())

    plots.append(p)

show(gridplot(plots, ncols=N))